---
# --- Day 19: Aplenty ---
---

In [1]:
from typing_extensions import TypedDict
from typing import Optional
import re

## Load data

In [2]:
full_puzzle_data = True

In [3]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day19_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [4]:
class MachinePart(TypedDict):
    x: int
    m: int
    a: int
    s: int

In [5]:
class Rule:
    def __init__(self, rule_def: str):
        condition, self.outcome = rule_def.split(":")
        self.part_id = condition[0]
        self.multiplier = 1 if condition[1] == ">" else -1
        self.quantity = int(condition[2:])
        
    def execute_rule(self, part: MachinePart) -> Optional[str]:
        if (part[self.part_id] - self.quantity) * self.multiplier > 0:
            return self.outcome
        else:
            return None        

In [6]:
class Workflow:
    def __init__(self, rules_string: str):
        rules_data = rules_string.split(",")
        self.rules = [Rule(r) for r in rules_data[:-1]]
        self.final_outcome = rules_data[-1]        
    
    def execute_workflow(self, part: MachinePart) -> str:
        for r in self.rules:
            rule_outcome = r.execute_rule(part)
            if rule_outcome:
                return rule_outcome
        return self.final_outcome

In [7]:
workflows = dict()
for i, row in enumerate(data):
    if not row:
        i += 1
        break
    wid, rules_string = row.split("{")
    workflows[wid] = Workflow(rules_string[:-1])

In [8]:
parts = []
for row in data[i:]:
    v = list(map(int, re.findall(r"\d+", row)))
    parts.append(MachinePart(x=v[0],m=v[1],a=v[2],s=v[3]))

## --- Part One ---

In [9]:
sum_accepted = 0
for p in parts:
    out = workflows["in"].execute_workflow(p)
    while not out in ["A", "R"]:
        out = workflows[out].execute_workflow(p)
    if out == "A":
        sum_accepted += sum(p.values())

In [10]:
print(sum_accepted)

420739


## --- Part Two ---